In [3]:
import urllib
import urllib2
import oauth2

from bs4 import BeautifulSoup
import lxml
import string

import pandas as pd
import numpy as np

import pymysql
#from KAsql import query_SQL

In [4]:
def get_html(business_id):
    
    yelp_url = 'http://www.yelp.com/biz/'+business_id
    response = urllib2.urlopen(yelp_url)
    html = response.read()
    return html

In [5]:
def save_myhtml(html_filename, html):
    # Save the html file
    with open(html_filename, 'wb') as f:
        f.write(html)

In [6]:
def query_SQL(yelp_business_id):
    con = False
    rows = []
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            sql = 'USE yelpdata;'
            cur.execute(sql)
            sql = 'SELECT id FROM business WHERE yelp_id = "'+yelp_business_id+'";' 
            #print(sql)
            cur.execute(sql)
            rows = cur.fetchall()
            #print('business id from table: ',rows, type(rows))
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()
    return rows  

In [ ]:
# def query_SQL(yelp_business_id):
#     con = False
#     rows = []
#     try:
#         con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
#         with con:
#             cur = con.cursor()
#             sql = 'USE yelpdata;'
#             cur.execute(sql)
#             sql = 'SELECT id FROM business WHERE yelp_id = "'+yelp_business_id+'";' 
#             #print(sql)
#             cur.execute(sql)
#             rows = cur.fetchall()
#             #print('business id from table: ',rows, type(rows))
#     except pymysql.Error, e:
#         print "Error %d: %s" % (e.args[0],e.args[1])
#         sys.exit(1)
#     finally:
#         if con:
#             con.close()
#     return rows   

In [ ]:
def get_business(business_id):
    """Query the Business API by a business ID.
    Args:
        business_id (str): The ID of the business to query.
    Returns:
        dict: The JSON response from the request.
    """
    business_path = BUSINESS_PATH + business_id

    return request(API_HOST, business_path)

In [7]:
def format_html(string,int):
    # accepts a rating or date and returns a float btw 0.0 and 5.0, or date-string
    s = str(string)
    s = s.lstrip('[<meta content="')
    if int==0:
        s = float(s[0:3])#star ratings are always 3 characters long
    elif int==1:
        stop = s.find('"')#date lengths could vary
        s = s[0:stop]
    else: 
        print('Wrong integer given to format_rating. Must be 0 for rating or 1 for date: ',int)
    return s

In [8]:
# not working i don't know why
def remove_backslash(review):
    #review = review.replace(u'\xa0','')
    review = review.replace(u"\\",'')#This doesn't work because in many cases it is a combination of characters with the backslash that represents a one-byte thing??
    review = review.replace(u"/",'')
    review = review.replace(u"(",'')
    review = review.replace(u")",'')
    review = review.replace(u"-",'')
    review = review.replace(u'"','')
    return review

In [9]:
def scrape_reviews(html_filename):

    # open the html file/object with beautiful soup
    
    soup = BeautifulSoup(open(html_filename), "lxml")

    #pp.pprint(ptext)#<class 'bs4.element.ResultSet'>
    ptext = soup.find_all('p', {'itemprop': 'description'})
    reviews = [ptext[i].get_text() for i in range(0,len(ptext))]
    reviews = [filter(lambda x: x in string.printable, review) for review in reviews]
    reviews = [remove_backslash(review) for review in reviews]
    
#     bads = [u'\xa0',"\\"]
#     reviews = [[review.replace(bad,'') for review in reviews]
#                for bad in bads]
    
    # Examples of what the html looks like:
    # <meta itemprop="ratingValue" content="5.0">
    # <meta itemprop="datePublished" content="2013-03-27">

    rstars =[];rdates=[]
    rstars = [pt.parent.find_all('meta', {'itemprop':'ratingValue'}) for pt in ptext]
    rdates = [pt.parent.find_all('meta', {'itemprop':'datePublished'}) for pt in ptext]
    rstars =[format_html(rstar,0) for rstar in rstars]
    rdates =[format_html(rdate,1) for rdate in rdates]

    # There's an overall rating on this page I could also get, but it should match bstars

    # Ensure that ratings and reviews are vectors of the same length
    if len(rstars)!=len(reviews):
        print('Length of rstars and reviews for ',html_filename[24:],' on this page do not match.')
        #print('rstars len: ',len(rstars),' reviews len: ',len(reviews))
        rmin = min(len(rstars),len(reviews))
        rstars = rstars[0,rmin]; reviews = reviews[0,rmin]; rdates = rdates[0,rmin]

    #Here I manually verified that the star-ratings, reviews, and dates match up with a few pages
#     for i in range(0,len(reviews)):
#         print(rstars[i],rdates[i],reviews[i])
        
    return [rstars], [reviews]

In [10]:
def insert_reviews_SQL(business_id, rstar, review):
    con = False
    try:
        con = pymysql.connect(host='localhost', port=3307, user='root', passwd='', db='yelpdata')
        with con:
            cur = con.cursor()
            #print('inserting into review db')
            #print(type(business_id), type(rstar), type(review))
            myvalues = "'" + str(business_id)+"'," + str(rstar) + "," + '"'+review+'"'
            sql = 'INSERT INTO review(business_id,stars,comment) VALUES('+myvalues+')'
            #print(sql)
            cur.execute(sql)
    except pymysql.Error, e:
        print "Error %d: %s" % (e.args[0],e.args[1])
        sys.exit(1)
    finally:
        if con:
            con.close()

In [13]:
def main():
    
    for i in range(0,20):# loop over 20 businesses on each YELP API query
    #for i in range(0,1):# for debugging: loop over 1 business for each YELP API query
    business_id = query_SQL(yelp_business_id)#business id is a tuple in a tuple

    print 'business id = ',business_id[0][0]

    # Save a temporary html file: soup breaks if you pass it html obj
    #html_filename = yelp_business_id +'.html'
    html = get_html(yelp_business_id)#returns an html page
    html_filename = 'temp.html'
    save_myhtml(html_filename, html)

    #Get reviews & their stars - Beautiful Soup scrape
    rstars,reviews = scrape_reviews(html_filename)

    for i in range(0,np.shape(rstars)[1]):#np.shape of rstars is (1,40)
    #for i in range(0,1):#np.shape of rstars is (1,40)
        #print('SQL review input: ',business_id[0], rstars[0][i],reviews[0][i[:10]])
        insert_reviews_SQL(business_id[0][0], rstars[0][i], reviews[0][i])
        
        
        
#         business_id = query_SQL(yelp_business_id)#business id is a tuple in a tuple
    
#         print('business id = ',business_id[0][0])

#         # Save a temporary html file: soup breaks if you pass it html obj
#         #html_filename = yelp_business_id +'.html'
#         html = get_html(yelp_business_id)#returns an html page
#         html_filename = 'temp.html'
#         save_myhtml(html_filename, html)

#         #Get reviews & their stars - Beautiful Soup scrape
#         rstars,reviews = scrape_reviews(html_filename)

#         for i in range(0,np.shape(rstars)[1]):#np.shape of rstars is (1,40)
#         #for i in range(0,1):#np.shape of rstars is (1,40)
#             #print('SQL review input: ',business_id[0], rstars[0][i],reviews[0][i[:10]])
#             insert_reviews_SQL(business_id[0][0], rstars[0][i], reviews[0][i])

            
            #x = np.asarray([[business_id]*40]).T
            #df = pd.DataFrame(rstars,reviews)
            #print(df)

#         DEGUGGING CODE ONLY
#         temp_url = 'total-care-plus-san-francisco.html'
#         temp_url = 'dan-kalshan-md-san-francisco-2.html'
    

if __name__ == '__main__':
   main()

NameError: name 'yelp_business_id' is not defined